In [17]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from collections import defaultdict
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import sys
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
TRAIN_DATA_FILE= 'gdrive/My Drive/dl/data/clean_train.csv'

train_df = pd.read_csv(TRAIN_DATA_FILE)
train_df = train_df.sample(frac=1).reset_index(drop=True)
print(train_df.columns)

Index(['comment_text', 'toxic'], dtype='object')


In [20]:
bad = set()
good = set()
all = set()



for text, label in zip(train_df['comment_text'][:120000],train_df['toxic'][:120000]):
    for word in text.split():
      all.add(word)
print(len(all))

for text, label in zip(train_df['comment_text'],train_df['toxic']):
  if(label == 1):
    for word in text.split():
      bad.add(word)
print(len(bad))
print(bad)
     
all = all - bad
print(len(all))
print(len(bad))
print(len(all) + len(bad))

132265
25296
{'unemploy', 'bleac', 'etc', 'offenc', 'idahoensi', 'exicut', 'kin', 'flaccid', 'somet', 'patrifuck', 'should', 'dipshit', 'ipso', 'driver', 'stage', 'neoconserv', 'neonazicommunist', 'media', 'buttpir', 'unsecur', 'deletesd', 'ahhahahah', 'kimimaro', 'makin', 'unquot', 'ark', 'sublimin', 'persuad', 'ist', 'intercourseoth', 'leaker', 'toho', 'fangirl', 'uberadmin', 'rott', 'wateva', 'fergalici', 'menj', 'oregon', 'morgu', 'centurion', 'incess', 'wank', 'slower', 'facet', 'thr', 'blighti', 'bc', 'ghalib', 'altruist', 'maa', 'lambeth', 'give', 'bison', 'cri', 'crafucka', 'kilrathi', 'loveh', 'evryday', 'bibi', 'ahm', 'kamehameha', 'poorer', 'thraten', 'load', 'illiter', 'covenantdtyp', 'rabbi', 'ceo', 'lfie', 'flavour', 'centuryfirst', 'bith', 'arsath', 'tvtime', 'oic', 'thurphftp', 'surciv', 'deobandi', 'express', 'peron', 'indirect', 'surind', 'especfi', 'wikiepia', 'cameroon', 'conciev', 'beem', 'greku', 'dead', 'inappropr', 'beggar', 'discust', 'permablofuck', 'tar', 'ca

In [21]:
data = pd.DataFrame()

data['word'] = list(all) + list(bad)
data['code'] = range(1, len(data['word']) + 1)

print(len(data['word']))
print(len(data['code']))

134052
134052


In [0]:
enc = {}
for word, code in zip(data['word'], data['code']):
  enc[word] = code

In [0]:
data.to_csv('gdrive/My Drive/dl/data/word_encodings.csv', index=False)

In [0]:
def transform(sentence):
  empty = '0'
  sent = ''
  i = 0
  for word in sentence.split():
    if word in enc and i < 300:
      sent += str(enc[word]) + ' '
      i += 1
      
  if i < 300:
    while i < 300:
      sent += empty + ' '
      i += 1
  return sent

In [25]:
train_df['comment_text'] = [transform(sent) for sent in train_df['comment_text']]
print('done')

max1 = max([len(sent.split()) for sent in train_df['comment_text']])
print(max1)
print([len(sent.split()) for sent in train_df['comment_text']])
train_df.to_csv('gdrive/My Drive/dl/data/sentences_encoded.csv', index=False)

done
300
[300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 

In [30]:
print(max([int(a) for a in [max(sent.split()) for sent in train_df['comment_text']]]))

134052
